In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

## Medida del CRPS
def heavyside(prediction,actual):
    return prediction >= actual

def cdf_dif(prediction,actual):
    quantiles = np.arange(1,100)/100.0
    t=pd.Series(prediction)
    dif=t-t.shift(1)
    dif=dif.dropna()
    fs = sum(dif*((quantiles-heavyside(prediction,actual))[1:]**2))
    # If the actual is outside the range of the prediction, 
    # we need to account for that areas outside the range 
    if actual > prediction[-1]:
        fs += (actual-prediction[-1]) * 1
    if actual < prediction[0]:
        fs += (prediction[0]-actual) * 1
    return fs

def CRPS(predictions, actuals):
    difs_mean = [cdf_dif(predictions[i],actuals[i]) for i in range(len(actuals))]
    return np.mean(difs_mean)

def evaluate(predictions,target):

    res={}
    
    # Calculate the CRPS
    res["crps"]=CRPS(predictions,target)
    
    # Bonus useful Feature
    count = 0
    for i in range(len(target)):
        if (target[i]>predictions[i][0]) and (target[i]<predictions[i][-1]):
            count+=1
    res["count"]=count
    
    ## Calculate as well measures for the quantile 50
    total_df = pd.DataFrame(predictions)
    quantiles = np.arange(1,100)/100.0 
    total_df.columns=np.array(quantiles).astype(str)
    #RMSE       
    res["rmse"]=np.sqrt(np.mean((target-total_df["0.5"])**2))
    #MAE    
    res["mae"]=np.mean(np.abs(target-total_df["0.5"] ) )
    #Bias 
    res["bias"]=np.mean(target-total_df["0.5"])
    #Corr
    res["corr"]=np.corrcoef(target,total_df["0.5"])[0][1]

    res["all"]=predictions
    res["target"]=target
    
    return res

In [45]:
import numpy as np
import math
import pandas as pd
from tqdm import  tqdm
from sklearn.model_selection import KFold
from numpy.fft import fft
import datetime
import json
import numpy as np
from scipy import stats

def prepare_data_from_horizon(df, horizon=12):
    
    data=df.copy()

    sel = np.concatenate([[1,2,3,4],
                         [12],
                         24*np.arange(1,9),
                         12+24*np.arange(1,9)])  
    sel=np.concatenate([sel,sel-1,sel+1])  
    
    ## lagged NO2 values
    for i in sel:
        if (i>=horizon):
            data["NO2 - "+str(i)] = data["NO2"].shift(i)

    ## lagged O3 values
    for i in 24*np.arange(1,4):
        if (i>=horizon):
            data["O3 - "+str(i)] = data["O3"].shift(i)

    ## Remove empty values
    data=data.dropna()

    X=data[list(set(data.columns)-set(['DATE',"NO2","O3","day","dt_date"]))]
    y=data["NO2"]

    return X, y

#"data/dataEscAgui.csv"
def get_data(path):

    df = pd.read_csv(path,sep=";")

    # Prepare data
    data = df[["DATE","SPA.NO2","SPA.O3","MACC.NO2"]].copy()
    data["DATE"]=pd.to_datetime(data["DATE"],format="%Y-%m-%d %H:%M:%S")
    data = data.sort_values("DATE")
    data.columns = ["DATE","NO2","O3","CAMS"]

    ## Remove everything from 2020
    data=data[data["DATE"].astype(str)<"2020"]

    ## Fourier Columns
    freqs = [2922,1461,209,1465,4]
    l = 35064
    n = np.arange(len(data))
    fcols = []
    for f in freqs:
        data["c"+str(f)]=np.cos(n*2*np.pi*f/l)
        fcols.append("c"+str(f))
        data["s"+str(f)]=np.cos(n*2*np.pi*f/l)
        fcols.append("s"+str(f))

    data["NO2"]=np.log1p(data["NO2"])
    data["O3"]=np.log1p(data["O3"])
    data["CAMS"]=np.log1p(data["CAMS"])

    ## Calendar Variables 
    ## # Day
    data["day"]=data["DATE"].dt.strftime("%Y-%m-%d")

    from datetime import date, timedelta, datetime

    def get_alldates(starts,ends):
        sdate = datetime.strptime(starts,"%Y-%m-%d")
        edate = datetime.strptime(ends,"%Y-%m-%d") 

        delta = edate - sdate       # as timedelta

        all_dates = []
        for i in range(delta.days + 1):
            day = sdate + timedelta(days=i)
            all_dates.append(day.strftime("%Y-%m-%d"))

        return all_dates

    # Escolar

    cal_escolar = pd.read_csv("../cal_extract/cal_f_escolar.csv")

    ## No cogemos los dias con COVID 19
    cal_escolar = cal_escolar[~cal_escolar["co_period"].str.contains("covid19")]

    all_cal_days = []
    for i,row in cal_escolar.iterrows():
        all_dates = get_alldates(row["dt_start"],row["dt_stop"])

        def process_date(date,typed):
            return {"day":date,typed:"1"}

        for element in map(lambda date: process_date(date,"Calendar.School."+row["co_type"]+"."+row["co_period"]),all_dates):
            all_cal_days.append(element)

    all_cal_days_df = pd.DataFrame(all_cal_days).fillna(0)
    
    for col in all_cal_days_df.columns:
        if col!="day":
            all_cal_days_df[col]=all_cal_days_df[col].astype(int)

    ## Otros Festivos
    """
    Calendar.Festivo.EqiNoc
    Calendar.Festivo.FesNov
    Calendar.Festivo.InmCns
    Calendar.Festivo.NavAnu
    Calendar.Festivo.PriVer
    Calendar.Festivo.SemSan
    Calendar.Festivo.VrgAgo
    Calendar.NocheNav.NocheBuena
    Calendar.NocheNav.NocheVieja
    Calendar.OprRetorno
    Calendar.OprSalida.PriNoLab
    Calendar.OprSalida.Vispera
    """ 
    cal_festivos = pd.read_csv("../cal_extract/cal_f_festivos.csv")
    
    
    ## For each type we take the OR of the columns
    holiday_list = """Festivo.EqiNoc
    Calendar.Festivo.FesNov
    Calendar.Festivo.InmCns
    Calendar.Festivo.NavAnu
    Calendar.Festivo.PriVer
    Calendar.Festivo.SemSan
    Calendar.Festivo.VrgAgo
    Calendar.NocheNav.NocheBuena
    Calendar.NocheNav.NocheVieja
    Calendar.OprRetorno
    Calendar.OprSalida.PriNoLab
    Calendar.OprSalida.Vispera""".replace("Calendar.","").lower().replace(".","_").split("\n")

    def get_festivos(name,row):
        cols = row.index[row.index.str.contains(name)]
        res = row[cols].sum()
        return res

    res_festivos_df = pd.DataFrame()
    for name in holiday_list:
        print(name)
        res_festivos_df["Calendar."+name]=cal_festivos.apply(lambda row:get_festivos(name,row),axis=1).astype(int).fillna(0)

    res_festivos_df["day"]=cal_festivos["dt_date"]
    
    data = data.merge(all_cal_days_df,on="day",how="left")
    data = data.merge(res_festivos_df,on="day",how="left")
    
    data=data.fillna(0)
    
    return data

In [56]:
import numpy as np
import math
import pandas as pd
from tqdm import tqdm
from scipy import stats

from keras.models import Sequential
from keras.layers import Dense, LeakyReLU
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
import keras.backend as K 

from sklearn.model_selection import train_test_split

def tilted_loss(q,y,f):
    e = (y-f)
    return K.mean(K.maximum(q*e, (q-1)*e), axis=-1)

class MLPQuantile():
    
    def __init__(self,batch_size,epochs,lr):
        self.batch_size = batch_size
        self.epochs = epochs
        self.lr=lr
        self.estimators = []
        
    def fit(self,X_train,y_train):
        
        def MLPmodel():
            model = Sequential()
            model.add(Dense(len(X_train[0]), input_dim=len(X_train[0]), activation=LeakyReLU(alpha=0.3)))
            model.add(Dense(int(len(X_train[0])/2), activation=LeakyReLU(alpha=0.3)))
            model.add(Dense(1, activation='linear'))
            return model
        
        print("training !")

        X_ttrain, X_val, y_ttrain, y_val = train_test_split(X_train,y_train,test_size=.05,random_state=2020)

        for q in [0.022750131948179195,0.15865525393145707,0.5,0.8413447460685429,0.9772498680518208]:
            print(f"Quantile: {q}")
            model = MLPmodel()
            optim=Adam(learning_rate=self.lr)
            model.compile(loss=lambda y,f: tilted_loss(q,y,f), optimizer=optim)
            es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=10)
            history = model.fit(X_ttrain, y_ttrain, 
                                epochs=self.epochs, batch_size=self.batch_size,  
                                verbose=1,callbacks=[es],
                                validation_data=(X_val,y_val))
            self.estimators.append(model)
        print("Done")
        
    def predict(self,X):
        predictions_gbr = []
        print("predicting")
        for reg in tqdm(self.estimators):
            predictions_gbr.append(reg.predict(X))
         
        total_pred={}
        for i in range(len(predictions_gbr)):
            total_pred[i]=predictions_gbr[i][:,0]
            
        total_df=pd.DataFrame(total_pred)

        def process_row(row):
            v = row.values
            dif_mean = np.abs(v-v[2])
            mu = v[2]
            s = np.mean([dif_mean[0]/2,dif_mean[1],dif_mean[3],dif_mean[4]/2])
            mi_norm = stats.norm(mu,s)
            quant=[]
            for quantile in np.arange(1,100)/100.0 :
                quant.append(mi_norm.ppf(quantile))
            return pd.Series(quant)
 
        total_df = total_df.apply(process_row,axis=1)
        
        return total_df.values


In [46]:
import lightgbm as lgb
import numpy as np
import math
import pandas as pd
from tqdm import tqdm
from scipy import stats

class TotalLGBQuantile():
    
    def __init__(self,n_estimators,max_depth):
        self.n_estimators=n_estimators
        self.max_depth=max_depth
        self.quantiles=[0.022750131948179195,0.15865525393145707,0.5,0.8413447460685429,0.9772498680518208]
        self.estimators = []
        
    def fit(self,X_train,y_train):
        print("training !")
        for q in tqdm(self.quantiles):
            print(f"Quantile: {q}")
            reg = lgb.LGBMRegressor(n_estimators=self.n_estimators,
                                    objective= 'quantile',
                                    loss="quantile",
                                    alpha=q,
                                    random_state=2020,
                                   max_depth=self.max_depth)
                                
            reg.fit(X_train, y_train)
            self.estimators.append(reg)
        print("Done")
        
    def predict(self,X):
        predictions_gbr = []
        print("predicting")
        for reg in tqdm(self.estimators):
            predictions_gbr.append(reg.predict(X))
         
        total_pred={}
        for i in range(len(predictions_gbr)):
            total_pred[i]=predictions_gbr[i]
            
        total_df=pd.DataFrame(total_pred)

        def process_row(row):
            v = row.values
            dif_mean = np.abs(v-v[2])
            mu = v[2]
            s = np.mean([dif_mean[0]/2,dif_mean[1],dif_mean[3],dif_mean[4]/2])
            mi_norm = stats.norm(mu,s)
            quant=[]
            for quantile in np.arange(1,100)/100.0 :
                quant.append(mi_norm.ppf(quantile))
            return pd.Series(quant)
 
        total_df = total_df.apply(process_row,axis=1)
        
        return total_df.values


In [47]:
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold

In [48]:
horizon = 13

df = get_data("../2018_2019_data/28079008.csv")
X, y = prepare_data_from_horizon(df,horizon)

          day Calendar.School.intensiva.inicio_curso  \
0  2011-09-05                                      1   
1  2011-09-06                                      1   
2  2011-09-07                                      1   
3  2011-09-08                                      1   
4  2011-09-09                                      1   

  Calendar.School.no_lectivo.otros Calendar.School.vacaciones.navidad  \
0                                0                                  0   
1                                0                                  0   
2                                0                                  0   
3                                0                                  0   
4                                0                                  0   

  Calendar.School.vacaciones.semana_santa Calendar.School.intensiva.fin_curso  \
0                                       0                                   0   
1                                       0                     

In [51]:
kf = KFold(5,shuffle=True)
train_index, test_index = list(kf.split(X))[1]

train_index = X.index.values[train_index]
test_index = X.index.values[test_index] 
        
# Filter the test index when prediction time is 10:00
ten_index = df[(df["DATE"]-timedelta(hours=horizon)).dt.hour==10].index
test_index_10 = test_index[pd.Series(test_index).isin(ten_index)]
        
# We retrieve the indexes that are related to the test indexes according to our AR model
sel = np.concatenate([[1,2,3,4],
                            [12],
                            24*np.arange(1,9),
                            12+24*np.arange(1,9)])  
sel=np.concatenate([sel,sel-1,sel+1]) 
        
all_index_related_test = set([])
for i in sel:
    all_index_related_test |= set(test_index_10+i)
        
train_index_CV = train_index[pd.Series(train_index).isin(list(all_index_related_test))]
        
X_train = X.loc[train_index_CV]
y_train = y.loc[train_index_CV]
        
X_test = X.loc[test_index_10]
y_test = y.loc[test_index_10]
        
scaler = RobustScaler()
# Fit the scaler on the training features and transform these in one go
X_train_std = scaler.fit_transform(X_train)
# Scale the test set
X_test_std = scaler.transform(X_test)
        
lin = LinearRegression()
lin.fit(X_train_std,y_train)

dif_train = y_train-lin.predict(X_train_std)
dif_test = y_test-lin.predict(X_test_std)

In [52]:
from tqdm.notebook import tqdm

In [49]:
alls=[]

for n_estimator in tqdm([500,600,700,800,900,1000,1100]):
    for max_depth in tqdm([4,6,8,10,12,14]):
        qreg = TotalLGBQuantile(n_estimators=n_estimator,max_depth=max_depth)
        qreg.fit(X_train_std,y_train)
        preds = qreg.predict(X_test_std)
        results=evaluate((np.exp(preds)-1),(np.exp(y_test)-1).values)
        del results["all"]
        del results["target"]
        results["n_estimator"]=n_estimator
        results["max_depth"]=max_depth
        alls.append(results)

training !


Quantile: 0.022750131948179195
[LightGBM] [Warning] Unknown parameter: loss
Quantile: 0.15865525393145707
Quantile: 0.5
Quantile: 0.8413447460685429
Quantile: 0.9772498680518208

Done
predicting


Exception ignored in: <function tqdm.__del__ at 0x11a1cf670>
Traceback (most recent call last):
  File "/Users/sebastien.perez/anaconda3/lib/python3.8/site-packages/tqdm/std.py", line 1122, in __del__
    self.close()
  File "/Users/sebastien.perez/anaconda3/lib/python3.8/site-packages/tqdm/notebook.py", line 261, in close
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'


training !


Quantile: 0.022750131948179195
[LightGBM] [Warning] Unknown parameter: loss
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Quantile: 0.15865525393145707
Quantile: 0.5
Quantile: 0.8413447460685429
Quantile: 0.9772498680518208

Done
predicting


...

Quantile: 0.022750131948179195
[LightGBM] [Warning] Unknown parameter: loss
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Quantile: 0.15865525393145707
Quantile: 0.5
Quantile: 0.8413447460685429
Quantile: 0.9772498680518208

Done
predicting



training !


Quantile: 0.022750131948179195
[LightGBM] [Warning] Unknown parameter: loss
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Quantile: 0.15865525393145707
Quantile: 0.5
Quantile: 0.8413447460685429
Quantile: 0.9772498680518208

Done
predicting



training !


Quantile: 0.022750131948179195
[LightGBM] [Warning] Unknown parameter: loss
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Quantile: 0.15865525393145707
Quantile: 0.5
Quantile: 0.8413447460685429
Quantile: 0.9772498680518208

Done
predicting


In [53]:
pd.DataFrame(alls).sort_values("rmse")

,crps,count,rmse,mae,bias,corr,n_estimator,max_depth
41,13.112044,465,26.195570,18.389143,2.557965,0.750437,1100,14
35,13.147471,465,26.276481,18.426206,2.578699,0.748572,1000,14
29,13.158772,466,26.331609,18.480324,2.641233,0.747443,900,14
39,13.168241,458,26.344224,18.464018,2.598883,0.746472,1100,10
33,13.189537,458,26.373749,18.490690,2.602618,0.745804,1000,10
23,13.170047,466,26.398462,18.535942,2.678638,0.745948,800,14
21,13.182804,461,26.418200,18.533462,2.567523,0.744575,800,10
27,13.253145,457,26.419564,18.516333,2.592532,0.744601,900,10
17,13.209801,466,26.457832,18.559086,2.721503,0.744805,700,14
15,13.173501,464,26.500201,18.589906,2.596564,0.742723,700,10


In [54]:
alls=[]

for n_estimator in tqdm([1500,1700,1900]):
    for max_depth in tqdm([16,18,20]):
        qreg = TotalLGBQuantile(n_estimators=n_estimator,max_depth=max_depth)
        qreg.fit(X_train_std,y_train)
        preds = qreg.predict(X_test_std)
        results=evaluate((np.exp(preds)-1),(np.exp(y_test)-1).values)
        del results["all"]
        del results["target"]
        results["n_estimator"]=n_estimator
        results["max_depth"]=max_depth
        alls.append(results)
pd.DataFrame(alls).sort_values("rmse")

training !


Quantile: 0.022750131948179195
[LightGBM] [Warning] Unknown parameter: loss
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Quantile: 0.15865525393145707
Quantile: 0.5
Quantile: 0.8413447460685429
Quantile: 0.9772498680518208

Done
predicting



training !


Quantile: 0.022750131948179195
[LightGBM] [Warning] Unknown parameter: loss
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Quantile: 0.15865525393145707
Quantile: 0.5
Quantile: 0.8413447460685429
Quantile: 0.9772498680518208

Done
predicting



training !


...


training !


Quantile: 0.022750131948179195
[LightGBM] [Warning] Unknown parameter: loss
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Quantile: 0.15865525393145707
Quantile: 0.5
Quantile: 0.8413447460685429
Quantile: 0.9772498680518208

Done
predicting



training !


Quantile: 0.022750131948179195
[LightGBM] [Warning] Unknown parameter: loss
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Quantile: 0.15865525393145707
Quantile: 0.5
Quantile: 0.8413447460685429
Quantile: 0.9772498680518208

Done
predicting


,crps,count,rmse,mae,bias,corr,n_estimator,max_depth
6,11.844544,506,24.586943,16.710684,2.939157,0.793963,1900,16
3,11.870154,506,24.656012,16.750462,2.935235,0.792500,1700,16
0,11.946960,505,24.744015,16.810188,2.940530,0.790726,1500,16
8,11.910498,508,24.929993,16.691116,2.852361,0.785449,1900,20
5,11.928698,508,24.949022,16.715360,2.907771,0.785276,1700,20
2,11.964930,508,25.027565,16.787508,2.903996,0.783670,1500,20
7,12.067769,508,25.037905,16.992165,2.818733,0.783291,1900,18
4,12.075730,508,25.050652,17.003337,2.819423,0.783022,1700,18
1,12.119178,508,25.100846,17.057005,2.832693,0.782127,1500,18


In [55]:
alls=[]

for n_estimator in tqdm([1500,1700,1900]):
    for max_depth in tqdm([16,18,20]):
        qreg = TotalLGBQuantile(n_estimators=n_estimator,max_depth=max_depth)
        
        qreg.fit(X_train,dif_train)
        pred_difs = qreg.predict(X_test)
        a=pd.DataFrame(pred_difs)
        for col in a.columns:
            a[col] += lin.predict(X_test_std) 
        
        results=evaluate((np.exp(a)-1).values,(np.exp(y_test)-1).values)
        del results["all"]
        del results["target"]
        results["n_estimator"]=n_estimator
        results["max_depth"]=max_depth
        alls.append(results)
pd.DataFrame(alls).sort_values("rmse")

training !


Quantile: 0.022750131948179195
[LightGBM] [Warning] Unknown parameter: loss
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Quantile: 0.15865525393145707
Quantile: 0.5
Quantile: 0.8413447460685429
Quantile: 0.9772498680518208

Done
predicting



training !


Quantile: 0.022750131948179195
[LightGBM] [Warning] Unknown parameter: loss
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Quantile: 0.15865525393145707
Quantile: 0.5
Quantile: 0.8413447460685429
Quantile: 0.9772498680518208

Done
predicting



training !


...


training !


Quantile: 0.022750131948179195
[LightGBM] [Warning] Unknown parameter: loss
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Quantile: 0.15865525393145707
Quantile: 0.5
Quantile: 0.8413447460685429
Quantile: 0.9772498680518208

Done
predicting



training !


Quantile: 0.022750131948179195
[LightGBM] [Warning] Unknown parameter: loss
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Quantile: 0.15865525393145707
Quantile: 0.5
Quantile: 0.8413447460685429
Quantile: 0.9772498680518208

Done
predicting


,crps,count,rmse,mae,bias,corr,n_estimator,max_depth
8,12.305422,501,25.323187,17.419142,2.393561,0.775451,1900,20
5,12.365605,500,25.403507,17.478062,2.382649,0.773622,1700,20
2,12.430092,500,25.511866,17.565425,2.420305,0.771363,1500,20
7,12.478619,500,25.578052,17.493194,2.477631,0.770689,1900,18
4,12.559875,500,25.717836,17.597044,2.496593,0.767575,1700,18
1,12.584963,502,25.777273,17.665578,2.489691,0.766247,1500,18
6,12.630936,501,25.949323,17.885219,2.200429,0.760398,1900,16
3,12.693372,501,26.086942,17.973778,2.260202,0.757491,1700,16
0,12.754133,501,26.141296,18.032357,2.220521,0.756226,1500,16


In [57]:
alls=[]

for batch_size in tqdm([32,64,200,500,1000,2000]):
    for epochs in tqdm([200]):
        for lr in tqdm([0.0001,0.001,0.01,0.1]):

            qreg = MLPQuantile(batch_size=batch_size,epochs=epochs,lr=lr)
            qreg.fit(X_train_std,y_train)
            preds = qreg.predict(X_test_std)
            results=evaluate((np.exp(preds)-1),(np.exp(y_test)-1).values)

            del results["all"]
            del results["target"]
            results["batch_size"]=batch_size
            results["epochs"]=epochs
            results["lr"]=lr
            alls.append(results)


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

training !
Quantile: 0.022750131948179195
Epoch 1/200
377/377 [==============================] - 1s 2ms/step - loss: 0.0716 - val_loss: 0.0467
Epoch 2/200
377/377 [==============================] - 0s 945us/step - loss: 0.0446 - val_loss: 0.0365
Epoch 3/200
377/377 [==============================] - 0s 949us/step - loss: 0.0364 - val_loss: 0.0310
Epoch 4/200
377/377 [==============================] - 0s 998us/step - loss: 0.0324 - val_loss: 0.0300
Epoch 5/200
377/377 [==============================] - 0s 948us/step - loss: 0.0300 - val_loss: 0.0289
Epoch 6/200
377/377 [==============================] - 0s 889us/step - loss: 0.0284 - val_loss: 0.0272
Epoch 7/200
377/377 [==============================] - 0s 926us/step - loss: 0.0267 - val_loss: 0.0254
Epoch 8/200
377/377 [==============================] - 0s 921us/step - loss: 0.0260 - val_loss: 0.0254
Epoch 9/200
377/377 [==============================] - 0s 877us/step - loss: 0.0250 - val_loss: 0.0247
Epoch 10/200
377/377 [===========

377/377 [==============================] - 0s 1ms/step - loss: 0.0792 - val_loss: 0.0848
Epoch 36/200
377/377 [==============================] - 0s 1ms/step - loss: 0.0776 - val_loss: 0.0851
Epoch 37/200
377/377 [==============================] - 0s 1ms/step - loss: 0.0771 - val_loss: 0.0849
Epoch 38/200
377/377 [==============================] - 0s 861us/step - loss: 0.0772 - val_loss: 0.0857
Epoch 39/200
377/377 [==============================] - 0s 991us/step - loss: 0.0772 - val_loss: 0.0842
Epoch 40/200
377/377 [==============================] - 0s 938us/step - loss: 0.0754 - val_loss: 0.0836
Epoch 41/200
377/377 [==============================] - 0s 948us/step - loss: 0.0774 - val_loss: 0.0838
Epoch 42/200
377/377 [==============================] - 0s 903us/step - loss: 0.0758 - val_loss: 0.0836
Epoch 43/200
377/377 [==============================] - 0s 895us/step - loss: 0.0748 - val_loss: 0.0831
Epoch 44/200
377/377 [==============================] - 0s 1ms/step - loss: 0.0755 

377/377 [==============================] - 0s 1ms/step - loss: 0.1251 - val_loss: 0.1406
Epoch 39/200
377/377 [==============================] - 0s 1ms/step - loss: 0.1243 - val_loss: 0.1396
Epoch 40/200
377/377 [==============================] - 0s 992us/step - loss: 0.1244 - val_loss: 0.1386
Epoch 41/200
377/377 [==============================] - 0s 1ms/step - loss: 0.1255 - val_loss: 0.1385
Epoch 42/200
377/377 [==============================] - 0s 899us/step - loss: 0.1250 - val_loss: 0.1372
Epoch 43/200
377/377 [==============================] - 0s 925us/step - loss: 0.1227 - val_loss: 0.1372
Epoch 44/200
377/377 [==============================] - 0s 868us/step - loss: 0.1239 - val_loss: 0.1366
Epoch 45/200
377/377 [==============================] - 0s 876us/step - loss: 0.1225 - val_loss: 0.1369
Epoch 46/200
377/377 [==============================] - 0s 867us/step - loss: 0.1226 - val_loss: 0.1376
Epoch 47/200
377/377 [==============================] - 0s 862us/step - loss: 0.122

377/377 [==============================] - 0s 889us/step - loss: 0.0721 - val_loss: 0.0853
Epoch 48/200
377/377 [==============================] - 0s 865us/step - loss: 0.0724 - val_loss: 0.0850
Epoch 49/200
377/377 [==============================] - 0s 869us/step - loss: 0.0729 - val_loss: 0.0832
Epoch 50/200
377/377 [==============================] - 0s 868us/step - loss: 0.0730 - val_loss: 0.0851
Epoch 51/200
377/377 [==============================] - 0s 882us/step - loss: 0.0721 - val_loss: 0.0856
Epoch 52/200
377/377 [==============================] - 0s 853us/step - loss: 0.0725 - val_loss: 0.0849
Epoch 53/200
377/377 [==============================] - 0s 929us/step - loss: 0.0724 - val_loss: 0.0843
Epoch 54/200
377/377 [==============================] - 0s 878us/step - loss: 0.0710 - val_loss: 0.0839
Epoch 55/200
377/377 [==============================] - 0s 879us/step - loss: 0.0716 - val_loss: 0.0831
Epoch 56/200
377/377 [==============================] - 0s 880us/step - loss:

...

7/7 [==============================] - 0s 10ms/step - loss: 0.0977 - val_loss: 0.0876
Epoch 23/200
7/7 [==============================] - 0s 9ms/step - loss: 0.0943 - val_loss: 0.0841
Epoch 24/200
7/7 [==============================] - 0s 9ms/step - loss: 0.0852 - val_loss: 0.0886
Epoch 25/200
7/7 [==============================] - 0s 8ms/step - loss: 0.0881 - val_loss: 0.0860
Epoch 26/200
7/7 [==============================] - 0s 8ms/step - loss: 0.0852 - val_loss: 0.0817
Epoch 27/200
7/7 [==============================] - 0s 8ms/step - loss: 0.0848 - val_loss: 0.0842
Epoch 28/200
7/7 [==============================] - 0s 9ms/step - loss: 0.0849 - val_loss: 0.0802
Epoch 29/200
7/7 [==============================] - 0s 8ms/step - loss: 0.0818 - val_loss: 0.0835
Epoch 30/200
7/7 [==============================] - 0s 8ms/step - loss: 0.0826 - val_loss: 0.0817
Epoch 31/200
7/7 [==============================] - 0s 8ms/step - loss: 0.0838 - val_loss: 0.0971
Epoch 32/200
7/7 [==============




  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 00055: early stopping
Done
predicting





 40%|████      | 2/5 [00:00<00:00, 15.21it/s]


100%|██████████| 5/5 [00:00<00:00, 14.71it/s]
<ipython-input-57-a59834279e78>:10: RuntimeWarning: overflow encountered in exp
  results=evaluate((np.exp(preds)-1),(np.exp(y_test)-1).values)
 83%|████████▎ | 5/6 [21:02<04:12, 252.42s/it]


ValueError: operands could not be broadcast together with shapes (97,) (98,) 

In [58]:
pd.DataFrame(alls).sort_values("rmse")

,crps,count,rmse,mae,bias,corr,batch_size,epochs,lr
5,1.422962e+01,501,2.731237e+01,1.983951e+01,-3.657562e+00,0.734871,64,200,0.0010
14,1.377067e+01,520,2.731568e+01,1.916355e+01,2.974284e+00,0.732003,500,200,0.0100
9,1.349487e+01,509,2.783379e+01,1.915386e+01,4.290892e+00,0.723289,200,200,0.0010
4,1.413734e+01,508,2.785583e+01,1.978315e+01,-3.097706e-02,0.716947,64,200,0.0001
13,1.385494e+01,516,2.826727e+01,1.937147e+01,-7.111256e-01,0.711382,500,200,0.0010
0,1.401178e+01,514,2.841920e+01,1.964609e+01,1.177599e+00,0.700589,32,200,0.0001
2,1.463244e+01,524,2.845343e+01,2.011684e+01,6.537401e-02,0.699538,32,200,0.0100
17,1.473368e+01,512,2.851783e+01,2.037366e+01,-4.468434e+00,0.719861,1000,200,0.0010
8,1.417941e+01,515,2.882323e+01,2.029484e+01,1.163991e+00,0.691672,200,200,0.0001
18,1.455770e+01,524,2.969299e+01,2.015031e+01,2.965062e+00,0.674825,1000,200,0.0100
